In [3]:
from transformers import AutoModel, AutoTokenizer

In [2]:
import torch

model_ckpt = "distilbert-base-uncased"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModel.from_pretrained(model_ckpt).to(device)

model.safetensors: 100%|██████████| 268M/268M [00:03<00:00, 70.2MB/s] 


In [5]:
text = "this is a test"

model_ckpt = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)   # 자동 클래스

inputs = tokenizer(text, return_tensors="pt")
print(f"input tensor size : {inputs['input_ids'].size()}")

input tensor size : torch.Size([1, 6])


In [6]:
inputs = {k:v.to(device) for k,v in inputs.items()}
with torch.no_grad():
    outputs = model(**inputs)
outputs

BaseModelOutput(last_hidden_state=tensor([[[-0.1565, -0.1862,  0.0528,  ..., -0.1188,  0.0662,  0.5470],
         [-0.3575, -0.6484, -0.0618,  ..., -0.3040,  0.3508,  0.5221],
         [-0.2772, -0.4459,  0.1818,  ..., -0.0948, -0.0076,  0.9958],
         [-0.2841, -0.3917,  0.3753,  ..., -0.2151, -0.1173,  1.0526],
         [ 0.2661, -0.5094, -0.3180,  ..., -0.4203,  0.0144, -0.2149],
         [ 0.9441,  0.0112, -0.4714,  ...,  0.1439, -0.7288, -0.1619]]]), hidden_states=None, attentions=None)

In [8]:
outputs.last_hidden_state.size()    # [batch_size, n_tokens, hidden_dim]

torch.Size([1, 6, 768])

In [9]:
outputs.last_hidden_state[:,0].size()   # [CLS]

torch.Size([1, 768])

In [10]:
tokenizer.model_input_names

['input_ids', 'attention_mask']

In [11]:
def extract_hidden_states(batch):
    inputs = {k:v.to(device) for k,v in batch.items() if k in tokenizer.model_input_names}
    
    with torch.no_grad():
        last_hidden_state = model(**inputs).last_hidden_state

    return {"hidden_state": last_hidden_state[:,0].cpu().numpy()}